In [1]:
# Import utils
import numpy as np
import pandas as pd
import seaborn as sns
import copy
import time
import datetime as dt
import pickle
import pyreadr
import json
import pickle
from joblib import dump, load, Parallel, delayed
import os

In [2]:
### Function to get predictions
def getMeanPred(y_samples,
                y_samples_SKU,
                ID_samples,
                ID_samples_SKU,
                weights_ij_global,
                weights_ij_local,
                params):

    # Get T horizon and R horizon rolling
    T_horizon = params['T_horizon']
    T_horizon_rolling = params['T_horizon_rolling']
    
    # Initialize
    xi_hats_wsaa_global = []
    xi_hats_wsaa_local = []
    xi_hats_saa = []
    xi_acts = []

    ## Iterate over full time horizon
    for t_current in range(T_horizon):   

        # Set current model params
        model_params = dict({
            'T': min(T_horizon_rolling,T_horizon-t_current),
            'N': int(params['sampleUpTo_start']+t_current)
        })    
        
        # Samples of unknown variable (i,t)
        xi_hat_global = np.array(
            y_samples.iloc[list(ID_samples.sale_yearweek < 
                           ID_samples_SKU.loc[model_params['N']].sale_yearweek),
                                         0:model_params['T']])
        
        xi_hat_local = np.array(
            y_samples_SKU.iloc[0:model_params['N'], 0:model_params['T']])
        
        xi_act = np.array(y_samples_SKU.iloc[model_params['N'],0:model_params['T']])

        # Sample weights (i)
        w_hat_global = np.array(weights_ij_global[t_current+1])
        w_hat_local = np.array(weights_ij_local[t_current+1])

        # Summarize weights and samples - global
        xi_hat_global = xi_hat_global[list(w_hat_global > 0)]
        w_hat_global = w_hat_global[list(w_hat_global > 0)]

        df_global=pd.DataFrame(data=np.hstack(
            (w_hat_global.reshape(w_hat_global.shape[0],1), xi_hat_global)), 
                columns=[-1] + list(range(0,model_params['T']))).groupby(
            list(range(0,model_params['T']))).agg(
                w_hat_global = (-1, np.sum)).reset_index()

        xi_hat_global = np.array(df_global[list(range(0,model_params['T']))])
        w_hat_global = np.array([df_global.w_hat_global]).flatten()
        
        # Summarize weights and samples - global
        xi_hat_local = xi_hat_local[list(w_hat_local > 0)]
        w_hat_local = w_hat_local[list(w_hat_local > 0)]

        df_local=pd.DataFrame(data=np.hstack(
            (w_hat_local.reshape(w_hat_local.shape[0],1), xi_hat_local)), 
                columns=[-1] + list(range(0,model_params['T']))).groupby(
            list(range(0,model_params['T']))).agg(
                w_hat_local = (-1, np.sum)).reset_index()

        xi_hat_local = np.array(df_local[list(range(0,model_params['T']))])
        w_hat_local = np.array([df_local.w_hat_local]).flatten()

        # Predict Weighted SAA
        xi_hats_wsaa_global = xi_hats_wsaa_global + list(sum(xi_hat_global * w_hat_global[:, np.newaxis]))
        xi_hats_wsaa_local = xi_hats_wsaa_local + list(sum(xi_hat_local * w_hat_local[:, np.newaxis]))

        # Predict SAA
        xi_hats_saa = xi_hats_saa + list(sum(np.array(y_samples_SKU.iloc[0:model_params['N'],0:model_params['T']]) * 
            1/np.array(y_samples_SKU.iloc[0:model_params['N'],0:model_params['T']]).shape[0]))

        # Actuals
        xi_acts = xi_acts + list(xi_act)

    # Average demand predictions
    mean_xi_hat_wsaa_global = np.mean(np.array(xi_hats_wsaa_global))
    mean_xi_hat_wsaa_local = np.mean(np.array(xi_hats_wsaa_local))
    mean_xi_hat_saa = np.mean(np.array(xi_hats_saa))
    mean_xi_act = np.mean(np.array(xi_acts))
            
    # Return
    return mean_xi_hat_wsaa_global, mean_xi_hat_wsaa_local, mean_xi_hat_saa, mean_xi_act

In [9]:
### Function to run main script
def run(SKU_range, T_horizon_rolling_range):

    # Initialize results
    MeanPred_results = pd.DataFrame()

    # For each SKU
    for SKU in SKU_range:

        # Status
        print('SKU:', SKU)

        # For each rolling horizon
        for T_horizon_rolling in T_horizon_rolling_range:

            # Get weights
            with open(PATH_SAMPLES+'/SKU'+str(SKU)+'/Static/Weights'+str(T_horizon_rolling)+'/weights_global_ij.p', 'rb') as f:
                weights_ij_global = pickle.load(f)
            del f
            
            with open(PATH_SAMPLES+'/SKU'+str(SKU)+'/Static/Weights'+str(T_horizon_rolling)+'/weights_local_ij.p', 'rb') as f:
                weights_ij_local = pickle.load(f)
            del f

            # Get samples
            robj = pyreadr.read_r(PATH_SAMPLES+'/SKU'+str(SKU)+'/Static/TmpFiles'+str(T_horizon_rolling)+'/Y_samples_mv_k.RDS')
            y_samples_SKU = robj[None]

            robj = pyreadr.read_r(PATH_DATA+'/Y_Data_mv_NEW.RData')
            y_samples = robj['Y_Data_mv']

            robj = pyreadr.read_r(PATH_SAMPLES+'/SKU'+str(SKU)+'/Static/TmpFiles'+str(T_horizon_rolling)+'/ID_samples_k.RDS')
            ID_samples_SKU = robj[None]

            robj = pyreadr.read_r(PATH_DATA+'/ID_Data_NEW.RData')
            ID_samples = robj['ID_Data']

            # Get sampling
            robj = pyreadr.read_r(PATH_SAMPLES+'/SKU'+str(SKU)+'/Static/TmpFiles'+str(T_horizon_rolling)+'/sampleUpTo_start.RDS')
            sampleUpTo_start = robj[None].iloc[0,0]

            # Planning params
            params = dict({
                'SKU': SKU,
                'T_horizon': 13,
                'T_horizon_rolling': T_horizon_rolling,
                'sampleUpTo_start': sampleUpTo_start
            })

            # Get predictive performance
            result = getMeanPred(y_samples,
                                 y_samples_SKU,
                                 ID_samples,
                                 ID_samples_SKU,
                                 weights_ij_global,
                                 weights_ij_local,
                                 params)
            
            mean_xi_hat_wsaa_global, mean_xi_hat_wsaa_local, mean_xi_hat_saa, mean_xi_hat = result

            # Results
            MeanPred_result = pd.DataFrame({
                'SKU': [SKU],
                'T_horizon_rolling': [T_horizon_rolling],
                'meanPred_wsaa_global': [mean_xi_hat_wsaa_global],
                'meanPred_wsaa_local': [mean_xi_hat_wsaa_local],
                'meanPred_saa': [mean_xi_hat_saa],
                'meanAct': [mean_xi_hat]
            })


            # Store
            if not MeanPred_results.empty:
                MeanPred_results = MeanPred_results.append(MeanPred_result)   
            else:
                MeanPred_results = pd.DataFrame(MeanPred_result) 

        
    # Return
    return MeanPred_results

In [ ]:
PATH_DATA='/home/fesc/Data'
PATH_PARAMS='/home/fesc/Data/Params'
PATH_SAMPLES='/home/fesc/Data/Samples'
PATH_RESULTS='/home/fesc/Data/Results'
PATH_KERNELS='/home/fesc/Data/Kernels'

# SKUs
SKU_range=range(1,460+1)

# Rolling horizons
T_horizon_rolling_range=range(1,5+1)

# Run
MeanPred_results = run(SKU_range, T_horizon_rolling_range)

# Save result
MeanPred_results.to_csv(path_or_buf=PATH_RESULTS+'/MeanPred.csv' , sep=',')